# ESN-PdM Framework ML models
This notebook contains the implementation of the ESN-PdM framework's machine learning models. These models are developed using the `tensorflow` library and trained with a dataset specifically generated for this project. The data is located in the `dataset` folder of this repository and consists of several files, each corresponding to one of the four classes the models are trained to predict. The data was generated using a BMI270 IMU and an ESP32, hence the features include the raw accelerometer and gyroscope readings from the IMU.

### Imports, Constants and Converters
First, we import the necessary libraries and define some constants and converters that will be used throughout the notebook.

In [1]:
import numpy as np
import pandas as pd
import random
import json
import os

# Disable logs except for errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

INPUT_DATA_PATH = 'datos/'

LABELS = {
    "Good": 0,
    "Acceptable": 1,
    "Unacceptable": 2,
    "Bad": 3
}

# --- Accelerometer Constants ---
G_MS2 = 9.80665
MAX_INT_VALUE_SENSOR = 32768.0
ACC_RAW_TO_MS2 = (G_MS2 / MAX_INT_VALUE_SENSOR)
SENSOR_ACC_RANGE = 2 # 8g

# --- Gyroscope Constants ---
SENSOR_GYR_RANGE = 250.0
PI = 3.14159265359
GYR_RAW_TO_RADS = (PI / 180.0) / MAX_INT_VALUE_SENSOR

# --- Converters ---
convert_raw_acc_to_ms2 = lambda raw: (pow(2, SENSOR_ACC_RANGE + 1) * ACC_RAW_TO_MS2) * raw
convert_raw_gyr_to_rads = lambda raw: SENSOR_GYR_RANGE * GYR_RAW_TO_RADS * raw

### Data Loading and Preprocessing
Next, we load the data and preprocess it. The data is loaded from the files in the `dataset` folder and preprocessed to be used in the models. The preprocessing steps include cleaning null values, converting the raw data to the appropriate format and normalizing the data.

In [2]:
with open(INPUT_DATA_PATH + 'column_names.json') as f:
    column_names = json.load(f)

labeled_data_frames = {l: [] for l in LABELS.values()}
for filename in os.listdir(INPUT_DATA_PATH):
    for label in LABELS.values():
        if filename.startswith(str(label)) and filename.endswith('.csv'):
            df = pd.read_csv(INPUT_DATA_PATH + filename, names=column_names, sep=';')
            
            # Convert the timestamp column to a datetime object
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            
            # Create a mask for rows where acc_x, acc_y, and acc_z are all 0
            mask_acc_zero = (df['acc_x'] == 0) & (df['acc_y'] == 0) & (df['acc_z'] == 0)

            # Create a mask for rows where gyro_x, gyro_y, and gyro_z are all 0
            mask_gyro_zero = (df['gyro_x'] == 0) & (df['gyro_y'] == 0) & (df['gyro_z'] == 0)

            # Combine the masks to identify rows where either condition is true
            mask_either_zero = mask_acc_zero | mask_gyro_zero

            # Filter out the rows from the DataFrame
            df = df[~mask_either_zero]

            # Convert the raw accelerometer data to m/s^2
            df['acc_x'] = df['acc_x'].apply(convert_raw_acc_to_ms2).astype("float32")
            df['acc_y'] = df['acc_y'].apply(convert_raw_acc_to_ms2).astype("float32")
            df['acc_z'] = df['acc_z'].apply(convert_raw_acc_to_ms2).astype("float32")

            # Convert the raw gyroscope data to rad/s
            df['gyro_x'] = df['gyro_x'].apply(convert_raw_gyr_to_rads).astype("float32")
            df['gyro_y'] = df['gyro_y'].apply(convert_raw_gyr_to_rads).astype("float32")
            df['gyro_z'] = df['gyro_z'].apply(convert_raw_gyr_to_rads).astype("float32")
            
            labeled_data_frames[label].append(df)

dataframes = {label: pd.concat(data_frames) for label, data_frames in labeled_data_frames.items()}

# Make a copy of all the dataframes, add a label column, and concatenate them into a single dataframe
labeled_data = pd.concat([df.assign(label=label) for label, df in dataframes.items()])
labeled_data.reset_index(drop=True, inplace=True)

# Print info and overall stats about the dataset
print(labeled_data.info())
print(labeled_data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572331 entries, 0 to 572330
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   id         572331 non-null  int64         
 1   sensor     572331 non-null  int64         
 2   acc_x      572331 non-null  float32       
 3   acc_y      572331 non-null  float32       
 4   acc_z      572331 non-null  float32       
 5   gyro_x     572331 non-null  float32       
 6   gyro_y     572331 non-null  float32       
 7   gyro_z     572331 non-null  float32       
 8   timestamp  572331 non-null  datetime64[ns]
 9   label      572331 non-null  int64         
dtypes: datetime64[ns](1), float32(6), int64(3)
memory usage: 30.6 MB
None
                  id         sensor          acc_x          acc_y  \
count  572331.000000  572331.000000  572331.000000  572331.000000   
mean    33363.663066       1.501467      -0.274573       0.085277   
min      1624.000000       1.00

### Sequence Generation and Data Splitting
After preprocessing the data, we generate sequences that will be used for training the models. These sequences are created by filling a buffer with `SEG_LENGTH` contiguous readings. Once the sequences are generated, we split the data into training, validation and test sets.

In [3]:
SEQ_LENGTH = 50

# crop dataframes to a number of rows that is a multiple of SEQ_LENGTH
for label, data in dataframes.items():
    dataframes[label] = data.iloc[:len(data) - len(data) % SEQ_LENGTH]

# Crear secuencias y etiquetas
sequences = []
labels = []

# Create sequences and labels
for label, data in dataframes.items():
    for i in range(0, len(data), SEQ_LENGTH):
        seq = data.iloc[i:i + SEQ_LENGTH]
        sequences.append(seq[['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']].values)
        labels.append(label)

sequences, labels = np.array(sequences), np.array(labels)

# Print the number of sequences to be fed into the model
print(f"Sequences: {sequences.shape}")
print(f"Labels: {labels.shape}")

# Split data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(sequences, labels, test_size=0.4, random_state=random.randint(0, 100))
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=random.randint(0, 100))

# Convert labels to categorical format
y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(LABELS))
y_val = tf.keras.utils.to_categorical(y_val, num_classes=len(LABELS))
y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(LABELS))

Sequences: (11444, 50, 6)
Labels: (11444,)


### Model Architecture
We define the model architecture, which consists of parallel `LSTM` and `Conv1D` layers followed by fully connected layers. The RNN layer operates on the sequence in the temporal domain, while the CNN layer operates on the sequence in the frequency domain by transforming the signal through `FFT`. This structure aims to capture both the temporal and spatial features of the data. The model is compiled using the Adam optimizer and the categorical cross-entropy loss function.

In [4]:
# Define the model
import tensorflow as tf

def create_model():
    # Input layer
    inputs = tf.keras.layers.Input(shape=(SEQ_LENGTH, 6))

    # Convolutional layers
    conv1 = tf.keras.layers.SeparableConv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
    conv2 = tf.keras.layers.SeparableConv1D(filters=64, kernel_size=3, activation='relu', padding='same')(conv1)
    pool = tf.keras.layers.MaxPooling1D()(conv2)
    pool_flat = tf.keras.layers.Flatten()(pool)

    # LSTM
    lstm = tf.keras.layers.LSTM(64, return_sequences=True)(inputs)
    lstm_flat = tf.keras.layers.Flatten()(lstm)

    # Concatenation of Conv and LSTM paths
    concat = tf.keras.layers.Concatenate()([pool_flat, lstm_flat])

    # Dense layers
    dense1 = tf.keras.layers.Dense(128, activation='relu')(concat)
    dense2 = tf.keras.layers.Dense(64, activation='relu')(dense1)
    dense3 = tf.keras.layers.Dense(32, activation='relu')(dense2)
    outputs = tf.keras.layers.Dense(len(LABELS), activation='softmax')(dense3)

    # Model creation
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model

### Cloud Model
The model to be deployed on the cloud layer of the ESN-PdM framework is trained using the training and validation sets. The model is trained for `50` epochs using mini-batches of size `32`. To avoid overfitting, we use early stopping with a patience of `5` epochs, i.e., if the validation loss does not improve for `5` consecutive epochs, the training is stopped. The model is saved to a file named `cloud_model.keras`.

In [5]:
# Training constants
CLOUD_MODEL_TRAIN_EPOCHS = 15
MAX_EPOCHS_WITHOUT_IMPROVEMENT = 5
BATCH_SIZE = 64

# Create the model
cloud_model = create_model()

# Compile the model
cloud_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=MAX_EPOCHS_WITHOUT_IMPROVEMENT, restore_best_weights=True)

# Train the model
cloud_model.fit(X_train, y_train, epochs=CLOUD_MODEL_TRAIN_EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluate the model
y_true = np.argmax(y_test, axis=1)
cloud_y_pred = cloud_model.predict(X_test)
cloud_y_pred_classes = np.argmax(cloud_y_pred, axis=1)
cloud_model_accuracy = np.mean(cloud_y_pred_classes == y_true)

# Save the model
cloud_model.save('cloud_model.keras')

Epoch 1/15
108/108 [==============================] - 4s 26ms/step - loss: 0.3918 - accuracy: 0.8522 - val_loss: 0.1527 - val_accuracy: 0.9537
Epoch 2/15
108/108 [==============================] - 3s 25ms/step - loss: 0.0819 - accuracy: 0.9747 - val_loss: 0.0797 - val_accuracy: 0.9755
Epoch 3/15
108/108 [==============================] - 3s 25ms/step - loss: 0.0543 - accuracy: 0.9819 - val_loss: 0.0678 - val_accuracy: 0.9808
Epoch 4/15
108/108 [==============================] - 3s 24ms/step - loss: 0.0296 - accuracy: 0.9913 - val_loss: 0.0632 - val_accuracy: 0.9825
Epoch 5/15
108/108 [==============================] - 3s 26ms/step - loss: 0.0182 - accuracy: 0.9943 - val_loss: 0.0724 - val_accuracy: 0.9808
Epoch 6/15
108/108 [==============================] - 3s 25ms/step - loss: 0.0200 - accuracy: 0.9942 - val_loss: 0.1053 - val_accuracy: 0.9690
Epoch 7/15
108/108 [==============================] - 3s 26ms/step - loss: 0.0151 - accuracy: 0.9953 - val_loss: 0.0769 - val_accuracy: 0.9830

/Users/rdelaf/Documents/thesis/edge-sensor-network/esn-cloud-layer/model/venv/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
# Print classification report
print("--- Classification Report ---")
print(classification_report(y_true, cloud_y_pred_classes, target_names=LABELS.keys()))

# Print confusion matrix
print("--- Confusion Matrix ---")
print(confusion_matrix(y_true, cloud_y_pred_classes))

# Print model accuracy
print(f"Model accuracy: {cloud_model_accuracy}")

# Print model summary
cloud_model.summary()

--- Classification Report ---
              precision    recall  f1-score   support

        Good       0.98      0.98      0.98       517
  Acceptable       0.98      1.00      0.99       635
Unacceptable       0.98      0.97      0.98       748
         Bad       0.97      0.99      0.98       389

    accuracy                           0.98      2289
   macro avg       0.98      0.98      0.98      2289
weighted avg       0.98      0.98      0.98      2289

--- Confusion Matrix ---
[[505   0   8   4]
 [  1 632   2   0]
 [  9  10 723   6]
 [  2   0   2 385]]
Model accuracy: 0.9807776321537789
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 50, 6)]              0         []                            
                                                                                      

### Gateway Model
Unlike a cloud server, a gateway, such as the Raspberry Pi 4 used in the ESN-PdM framework, has limited computational resources. Therefore, the model deployed on the gateway layer needs to be lightweight yet accurate. To achieve this, we **fine-tune the pre-trained cloud model using pruning**. Pruning is a technique that removes less important weights from the model, reducing its size and computational complexity. During the fine-tuning process, the pruning sparsity is gradually increased from `0.50` to `0.80`. Pruning sparsity is the ratio of the number of weights zeroed out to the total number of weights in the model. In other words, after pruning, the resulting model will have 80% of its weights zeroed out.

The pruned model is then converted to a TensorFlow Lite model, optimized for deployment on edge devices. Among these optimizations is **post-training quantization**, which further reduces the model's size and computational complexity by converting its parameters from 32-bit floating-point numbers to 8-bit integers. Specifically, the type of quantization used for the gateway model is **dynamic range quantization**. This type of quantization only affects the weights of the model, leaving the activations in floating-point format.

The final model is saved to a file named `gateway_model.tflite`. Note that applying a standard compression algorithm, such as gzip, is necessary to fully realize the compression benefits of pruning.

In [7]:
import tensorflow_model_optimization as tfmot
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

GATEWAY_FINE_TUNE_EPOCHS = 3
FINE_TUNE_EPOCHS_WITHOUT_IMPROVEMENT = 1

num_sequences = X_train.shape[0]
end_step = np.ceil(num_sequences / BATCH_SIZE).astype(np.int32) * GATEWAY_FINE_TUNE_EPOCHS
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0,
        final_sparsity=0.50,
        begin_step=0,
        end_step=end_step
    )
}

callbacks = [
  tf.keras.callbacks.EarlyStopping(
      monitor='val_accuracy',
      patience=FINE_TUNE_EPOCHS_WITHOUT_IMPROVEMENT,
      restore_best_weights=False
  ),
  tfmot.sparsity.keras.UpdatePruningStep(),
]

# Load the cloud model and fine-tune it with pruning
_loaded_cloud_model = tf.keras.models.load_model('cloud_model.keras')
gateway_model = prune_low_magnitude(_loaded_cloud_model, **pruning_params)

# Compile the model
gateway_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
gateway_model.fit(
    x=X_train,
    y=y_train,
    epochs=GATEWAY_FINE_TUNE_EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=callbacks
)



Epoch 1/3
108/108 [==============================] - 5s 29ms/step - loss: 0.3549 - accuracy: 0.8742 - val_loss: 0.2517 - val_accuracy: 0.9048
Epoch 2/3
108/108 [==============================] - 3s 27ms/step - loss: 0.1078 - accuracy: 0.9649 - val_loss: 0.1580 - val_accuracy: 0.9502
Epoch 3/3
108/108 [==============================] - 3s 26ms/step - loss: 0.0777 - accuracy: 0.9768 - val_loss: 0.0954 - val_accuracy: 0.9672


In [8]:
# Evaluate the model
y_true = np.argmax(y_test, axis=1)
gateway_y_pred = gateway_model.predict(X_test)
gateway_y_pred_classes = np.argmax(gateway_y_pred, axis=1)
gateway_model_accuracy = np.sum(y_true == gateway_y_pred_classes) / len(y_true)

# Print classification report
print("--- Classification Report ---")
print(classification_report(y_true, gateway_y_pred_classes, target_names=LABELS.keys()))

# Print confusion matrix
print("--- Confusion Matrix ---")
print(confusion_matrix(y_true, gateway_y_pred_classes))

# Print accuracy before tflite conversion
print(f"Gateway model accuracy: {gateway_model_accuracy}")

# Print the gateway model summary
gateway_model.summary()

72/72 [==============================] - 0s 4ms/step
--- Classification Report ---
              precision    recall  f1-score   support

        Good       0.95      0.96      0.96       517
  Acceptable       0.99      0.99      0.99       635
Unacceptable       0.98      0.97      0.98       748
         Bad       0.94      0.95      0.94       389

    accuracy                           0.97      2289
   macro avg       0.97      0.97      0.97      2289
weighted avg       0.97      0.97      0.97      2289

--- Confusion Matrix ---
[[496   1   7  13]
 [  1 630   4   0]
 [  6   3 729  10]
 [ 18   0   2 369]]
Gateway model accuracy: 0.9716033202271734
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 50, 6)]              0         []                            
                       

In [9]:
# Remove pruning wrappers from the pruned model
gateway_model = tfmot.sparsity.keras.strip_pruning(gateway_model)

# Make a Tensorflow Lite version of the model and save it
converter = tf.lite.TFLiteConverter.from_keras_model(gateway_model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]

# Dynamic range quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]  

bytes_gateway_model = converter.convert()
with open('gateway_model.tflite', 'wb') as f:
    f.write(bytes_gateway_model)

INFO:tensorflow:Assets written to: /var/folders/cv/np29wc5x57scn_0hl8328btr0000gn/T/tmpymmfptf8/assets


INFO:tensorflow:Assets written to: /var/folders/cv/np29wc5x57scn_0hl8328btr0000gn/T/tmpymmfptf8/assets


### Sensor Model
Finally, we train a model to be deployed on the ESP32 microcontroller. Given the minimal computational resources available on the ESP32, the model needs to be extremely lightweight. Therefore, we simplify the model architecture by employing only `Conv1D` layers and reducing the number of neurons in the fully connected layers. The optimizations for the sensor model go beyond those used for the gateway model. In addition to **pruning**, the model is quantized using **full integer quantization**. This type of quantization converts both the weights and activations of the model from 32-bit floating-point numbers to 8-bit integers. Full integer quantization is more aggressive than dynamic range quantization, but it requires calibrating the quantization parameters using a representative dataset. The calibration dataset is generated by sampling a subset of the training data. It is important to note that LSTM layers are not used in the sensor model because they are computationally expensive and are not included in TensorFlow Lite's `TFLITE_BUILTINS_INT8` operation set, which is required for full integer quantization.

In [10]:
SENSOR_MODEL_TRAIN_EPOCHS = 15
SENSOR_FINE_TUNE_EPOCHS = 5

# --- Sensor Model Training ---
print("Training sensor model...")
sensor_model = tf.keras.Sequential([
    # Input layer
    tf.keras.layers.Input(shape=(SEQ_LENGTH, 6)),
    
    # Convolutional layers
    tf.keras.layers.SeparableConv1D(filters=16, kernel_size=3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Flatten(),
    
    # Dense layers
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(len(LABELS), activation='softmax')
])

callbakcs = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=MAX_EPOCHS_WITHOUT_IMPROVEMENT, restore_best_weights=True)
]
sensor_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
sensor_model.fit(X_train, y_train, epochs=SENSOR_MODEL_TRAIN_EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val), callbacks=callbacks)

# --- Sensor Model Fine-Tuning ---
print("Fine-tuning sensor model...")
num_sequences = X_train.shape[0]
end_step = np.ceil(num_sequences / BATCH_SIZE).astype(np.int32) * SENSOR_FINE_TUNE_EPOCHS
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0,
        final_sparsity=0.60,
        begin_step=0,
        end_step=end_step
    )
}

fine_tuning_callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=MAX_EPOCHS_WITHOUT_IMPROVEMENT, restore_best_weights=False)
]

# Create a pruned model
sensor_model = prune_low_magnitude(sensor_model, **pruning_params)

# Compile the pruned model
sensor_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune with pruning
sensor_model.fit(X_train, y_train, epochs=SENSOR_FINE_TUNE_EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val), callbacks=fine_tuning_callbacks)

# Evaluate the model
y_true = np.argmax(y_test, axis=1)
sensor_y_pred = sensor_model.predict(X_test)
sensor_y_pred_classes = np.argmax(sensor_y_pred, axis=1)
sensor_model_accuracy = np.sum(y_true == sensor_y_pred_classes) / len(y_true)

Training sensor model...
Epoch 1/15
108/108 [==============================] - 0s 2ms/step - loss: 0.8728 - accuracy: 0.6717 - val_loss: 0.5244 - val_accuracy: 0.7951
Epoch 2/15
108/108 [==============================] - 0s 2ms/step - loss: 0.4253 - accuracy: 0.8300 - val_loss: 0.3817 - val_accuracy: 0.8366
Epoch 3/15
108/108 [==============================] - 0s 2ms/step - loss: 0.3218 - accuracy: 0.8774 - val_loss: 0.3100 - val_accuracy: 0.8842
Epoch 4/15
108/108 [==============================] - 0s 2ms/step - loss: 0.2452 - accuracy: 0.9125 - val_loss: 0.2669 - val_accuracy: 0.9043
Epoch 5/15
108/108 [==============================] - 0s 2ms/step - loss: 0.2043 - accuracy: 0.9292 - val_loss: 0.2245 - val_accuracy: 0.9358
Epoch 6/15
108/108 [==============================] - 0s 2ms/step - loss: 0.1666 - accuracy: 0.9482 - val_loss: 0.2175 - val_accuracy: 0.9358
Epoch 7/15
108/108 [==============================] - 0s 2ms/step - loss: 0.1456 - accuracy: 0.9528 - val_loss: 0.1895 - va

In [11]:
# Print classification report
print("--- Classification Report ---")
print(classification_report(y_true, sensor_y_pred_classes, target_names=LABELS.keys()))

# Print confusion matrix
print("--- Confusion Matrix ---")
print(confusion_matrix(y_true, sensor_y_pred_classes))

# Print accuracy before tflite conversion
print(f"Sensor model accuracy: {sensor_model_accuracy}")

# Print the sensor model summary
sensor_model.summary()

--- Classification Report ---
              precision    recall  f1-score   support

        Good       0.94      0.89      0.91       517
  Acceptable       0.97      0.99      0.98       635
Unacceptable       0.96      0.93      0.95       748
         Bad       0.91      0.98      0.94       389

    accuracy                           0.95      2289
   macro avg       0.94      0.95      0.95      2289
weighted avg       0.95      0.95      0.95      2289

--- Confusion Matrix ---
[[462   8  23  24]
 [  1 631   3   0]
 [ 24  14 697  13]
 [  6   0   3 380]]
Sensor model accuracy: 0.9480122324159022
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_separa  (None, 50, 16)            228       
 ble_conv1d_4 (PruneLowMagn                                      
 itude)                                                          
                                         

In [12]:
# Remove pruning wrappers from the pruned model
sensor_model = tfmot.sparsity.keras.strip_pruning(sensor_model)

# Representative dataset for quantization
def representative_dataset():
    for i in range(0, X_train.shape[0], BATCH_SIZE):
        yield [X_train[i:i + BATCH_SIZE]]

# Make a Tensorflow Lite version of the model and save it
converter = tf.lite.TFLiteConverter.from_keras_model(sensor_model)

# Full integer quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.representative_dataset = representative_dataset
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

bytes_sensor_model = converter.convert()
with open('sensor_model.tflite', 'wb') as f:
    f.write(bytes_sensor_model)

INFO:tensorflow:Assets written to: /var/folders/cv/np29wc5x57scn_0hl8328btr0000gn/T/tmp8nuy8pon/assets


INFO:tensorflow:Assets written to: /var/folders/cv/np29wc5x57scn_0hl8328btr0000gn/T/tmp8nuy8pon/assets
/Users/rdelaf/Documents/thesis/edge-sensor-network/esn-cloud-layer/model/venv/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
fully_quantize: 0, inference_type: 6, input_inference_type: UINT8, output_inference_type: UINT8


### Model Comparison
Below me compare the performance of the cloud, gateway, and sensor models in terms of accuracy and size. The accuracy is evaluated using the test set, while the size is measured in terms of the compressed model file size.

In [13]:
# Size comparison
def get_gzipped_model_size(file):
  import os, zipfile, tempfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)
  return os.path.getsize(zipped_file)

def format_size(size_bytes):
    if size_bytes < 1024:
        return f"{size_bytes:.2f} bytes"
    elif size_bytes < 1024 ** 2:
        return f"{size_bytes / 1024:.2f} KB"
    else:
        return f"{size_bytes / 1024 ** 2:.2f} MB"

In [14]:
# Accuracy comparison
def _quantize_model(input_data, input_details):
    input_scale, input_zero_point = input_details["quantization"]
    input_data = input_data / input_scale + input_zero_point
    return input_data


def evaluate_tflite_model(tflite_model_path, X_test, y_test):
    # Load the TFLite model and allocate tensors
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    # Get input and output tensor details
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    correct_predictions = 0
    
    for i in range(X_test.shape[0]):
        # Check if the input type is quantized, then rescale input data to uint8
        input_data = _quantize_model(X_test[i], input_details) if input_details['dtype'] == np.uint8 else X_test[i]

        input_data = np.expand_dims(input_data, axis=0).astype(input_details['dtype'])   
        interpreter.set_tensor(input_details['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details['index'])
        
        predicted_label = np.argmax(output_data)


        true_label = np.argmax(y_test[i])

        if predicted_label == true_label:
            correct_predictions += 1

    return correct_predictions / X_test.shape[0]

In [15]:
X_train, X_temp, y_train, y_temp = train_test_split(sequences, labels, test_size=0.4, random_state=random.randint(0, 100))
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=random.randint(0, 100))

# Convert labels to categorical format
y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(LABELS))
y_val = tf.keras.utils.to_categorical(y_val, num_classes=len(LABELS))
y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(LABELS))

cloud_size = get_gzipped_model_size("cloud_model.keras")
gateway_size = get_gzipped_model_size("gateway_model.tflite")
sensor_size = get_gzipped_model_size("sensor_model.tflite")

tflite_gateway_model_accuracy = evaluate_tflite_model('gateway_model.tflite', X_test, y_test)
tflite_sensor_model_accuracy = evaluate_tflite_model('sensor_model.tflite', X_test, y_test)

INFO: Created TensorFlow Lite delegate for select TF ops.
INFO: TfLiteFlexDelegate delegate: 2 nodes delegated out of 27 nodes with 2 partitions.

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [16]:
# Print the model sizes
print("\n---------------------- Model Sizes ----------------------")
print("Size of cloud Keras model: %s" % format_size(os.path.getsize("cloud_model.keras")))
print("Size of gateway Tflite model: %s" % format_size(os.path.getsize("gateway_model.tflite")))
print("Size of sensor Tflite model: %s" % format_size(os.path.getsize("sensor_model.tflite")))

print("\n---------------------- Gzipped Model Sizes ----------------------")
print("Size of gzipped cloud Keras model: %s" % format_size(cloud_size))
print("Size of gzipped gateway Tflite model: %s" % format_size(gateway_size))
print("Size of gzipped sensor Tflite model: %s" % format_size(sensor_size))

# Print how much smaller the gateway model is
print("\n---------------------- Model Size Comparison ----------------------")
print("Gateway model is %.2f times smaller than the cloud model" % (get_gzipped_model_size("cloud_model.keras") / get_gzipped_model_size("gateway_model.tflite")))
print("Sensor model is %.2f times smaller than the cloud model" % (get_gzipped_model_size("cloud_model.keras") / get_gzipped_model_size("sensor_model.tflite")))
print("Sensor model is %.2f times smaller than the gateway model" % (get_gzipped_model_size("gateway_model.tflite") / get_gzipped_model_size("sensor_model.tflite")))

# Print the model accuracies
print("\n---------------------- Model Accuracies ----------------------")
print(f"Cloud model accuracy: {cloud_model_accuracy * 100:.2f}% [Tensorflow Keras]")
print(f"Gateway model accuracy: {tflite_gateway_model_accuracy * 100:.2f}% [TensorFlow Lite]")
print(f"Sensor model accuracy: {tflite_sensor_model_accuracy * 100:.2f}% [TensorFlow Lite Micro]")


---------------------- Model Sizes ----------------------
Size of cloud Keras model: 7.49 MB
Size of gateway Tflite model: 653.15 KB
Size of sensor Tflite model: 13.08 KB

---------------------- Gzipped Model Sizes ----------------------
Size of gzipped cloud Keras model: 6.87 MB
Size of gzipped gateway Tflite model: 395.83 KB
Size of gzipped sensor Tflite model: 7.92 KB

---------------------- Model Size Comparison ----------------------
Gateway model is 17.78 times smaller than the cloud model
Sensor model is 887.85 times smaller than the cloud model
Sensor model is 49.95 times smaller than the gateway model

---------------------- Model Accuracies ----------------------
Cloud model accuracy: 98.08% [Tensorflow Keras]
Gateway model accuracy: 97.34% [TensorFlow Lite]
Sensor model accuracy: 94.58% [TensorFlow Lite Micro]
